In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key
import ipywidgets as widgets

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
#city_data_df.head()

In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
#Get the latitidue and longitude of each city
locations = city_data_df[["Lat","Lng"]]
#set any negative temperatures to 0F (shouldn't it be better to use Kelvin which is an absolute temp?)
city_data_df["Max Temp"] = city_data_df["Max Temp"].clip(lower=0)
#Get the max temperatures of each city
max_temp = city_data_df["Max Temp"]
# assign the gmaps figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# assign the heatmap
heatmap_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating = False, 
                                    max_intensity=300,point_radius=4)
# Add the heatmap layer
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=clouds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
winds = city_data_df["Wind Speed"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=winds, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [10]:
temp_range_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Max Temp"] >= min_temp)]
temp_range_df = temp_range_df.dropna()
temp_range_df.head(5)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Montelibano,CO,2021-04-06 14:54:59,7.9829,-75.4229,82.24,75,100,6.04
4,4,Bandarbeyla,SO,2021-04-06 14:55:00,9.4942,50.8122,80.35,59,80,10.09
7,7,Miri,MY,2021-04-06 14:51:02,4.4148,114.0089,78.80,94,40,4.61
8,8,Vaini,TO,2021-04-06 14:55:01,-21.2000,-175.2000,71.60,94,75,5.75
11,11,Cape Town,ZA,2021-04-06 14:55:02,-33.9258,18.4232,77.00,41,0,9.22


In [11]:
temp_range_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 212 entries, 0 to 551
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     212 non-null    int64  
 1   City        212 non-null    object 
 2   Country     212 non-null    object 
 3   Date        212 non-null    object 
 4   Lat         212 non-null    float64
 5   Lng         212 non-null    float64
 6   Max Temp    212 non-null    float64
 7   Humidity    212 non-null    int64  
 8   Cloudiness  212 non-null    int64  
 9   Wind Speed  212 non-null    float64
dtypes: float64(4), int64(3), object(3)
memory usage: 18.2+ KB


In [12]:
hotel_df = temp_range_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Montelibano,CO,82.24,7.9829,-75.4229,
4,Bandarbeyla,SO,80.35,9.4942,50.8122,
7,Miri,MY,78.80,4.4148,114.0089,
8,Vaini,TO,71.60,-21.2000,-175.2000,
11,Cape Town,ZA,77.00,-33.9258,18.4232,
13,Eyl,SO,81.05,7.9803,49.8164,
14,Castro,BR,71.74,-24.7911,-50.0119,
18,Santa Rosa,PH,82.40,14.3122,121.1114,
19,Rikitea,PF,77.31,-23.1203,-134.9692,
22,Toccoa,US,73.40,34.5773,-83.3324,


In [13]:
def find_hotel(row) :
    lat = row["Lat"]
    lng = row["Lng"]
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params = {
        "radius": 5000,
        "types": "lodging",
        "key": g_key,
        "location": f"{lat},{lng}"}
    r = requests.get(base_url,params).json()
    try :
        row["Hotel Name"] = r["results"][0]["name"]
    except IndexError:
        print("Hotel not found... skipping")
    return row

hotel_df = hotel_df.apply(find_hotel,axis=1)
hotel_df.head()

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Montelibano,CO,82.24,7.9829,-75.4229,Montelíbano Plaza Hotel
4,Bandarbeyla,SO,80.35,9.4942,50.8122,JABIR HOTEL
7,Miri,MY,78.80,4.4148,114.0089,"Mega Hotel, Miri"
8,Vaini,TO,71.60,-21.2000,-175.2000,Keleti Beach Resort
11,Cape Town,ZA,77.00,-33.9258,18.4232,Southern Sun Waterfront Cape Town


In [14]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


In [15]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))